<img style="float: left; margin: 30px 15px 15px 15px;" src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFzQj91sOlkeDFkg5HDbjtR4QJYmLXkfMNig&usqp=CAU" width="400" height="500" /> 
    
    
### <font color='navy'> Modelos no lineales para pronósticos. 

**Nombres:** Andre Gonzalez, Emiliano Rodriguez

**Fecha:** 07 de Marzo del 2023.

**Expediente** : 739922, 738288
    
**Profesor:** Oscar David Jaramillo Zuluaga.
    
**Link Github**: https://github.com/Andr3Glez/Proyecto_conjunto3_CERodriguez_ANGonzalez-?query=Andr3Glez

# Tarea 4: Clase 11

# <font color='red'> Tarea: </font>

Terminar los demás numerales faltantes de este ejercicio.

Código de solución estudiante 1

In [ ]:
# Código de solución estudiante 1
.
.
.
.


Código de solución estudiante 2

In [ ]:
# Código de solución estudiante 2
.
.
.
.
